In [1]:
#Import libralies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
import random

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *

#You can import other libraries if you need

2023-12-14 07:50:28.827099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
###### DO NOT MODIFY THIS PART
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)  # fix your random seed : In same model, you produce the same result
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)  
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf) # tensorflow class for your operation 
tf.compat.v1.keras.backend.set_session(sess)
######

2023-12-14 07:50:41.703505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 07:50:43.273050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7918 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 1g.10gb, pci bus id: 0000:85:00.0, compute capability: 8.0


In [3]:
artists = pd.read_csv('./test/artists.csv') # type your directory
# Sort artists by number of paintings
artists = artists.sort_values(by=['paintings'], ascending=False)

# Create a dataframe 
artists_top = artists[artists['paintings'] >= 0].reset_index()
artists_top = artists_top[['name', 'paintings']]

updated_name = "Albrecht_Dürer".replace("_", " ")
artists_top.iloc[4, 0] = updated_name

In [5]:
#Test set load
images_dir = './test/images/' # type your directory
artists_dirs = os.listdir(images_dir)
artists_top_name = artists_top['name'].str.replace(' ', '_').values

input_shape = (128,128)#Same as input_shape in your train code
n_classes = artists_top.shape[0]

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=input_shape,
                                                    batch_size=1,
                                                    shuffle=False,
                                                    seed=seed,
                                                   )

Found 220 images belonging to 1 classes.


In [6]:
model = tf.keras.models.load_model("resnet_128x128.h5")

test_generator.reset()
output = model.predict(test_generator, steps=test_generator.n)
output = np.argmax(output, axis=-1)
prediction = pd.DataFrame(output, index=test_generator.filenames, columns=['predictions']).sort_index().to_csv('20201234_output.csv') # Type your student ID

2023-12-14 07:51:16.243709: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7918 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 1g.10gb, pci bus id: 0000:85:00.0, compute capability: 8.0
2023-12-14 07:51:19.803784: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-12-14 07:51:21.750101: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  6/220 [..............................] - ETA: 8s 

2023-12-14 07:51:22.280240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


220/220 [==============================] - 11s 31ms/step
